In [1]:
using DataFrames;

In [45]:
using LinearAlgebra;

In [46]:
using Printf;

In [3]:
TOL = 1e-4

0.0001

In [4]:
tensor_df = readtable("tensor_head.csv");
for i in 1:size(tensor_df,1)
    curr_row = tensor_df[i,1:9];
    T_flat = [x for x in curr_row]
    T = transpose(reshape(T_flat, 3, 3))
    println(T)
    println("*****")
end

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[4]:1
└ @ Core In[4]:1


[-11.317 -10.12 2.8978; 45.466 86.037 -433.36; 0.028671 38.248 -62.274]
*****
[163.09 29.24 -198.21; 122.05 21.069 -89.915; 457.34 76.642 -172.86]
*****
[-44.434 189.45 81.13; 78.246 -45.926 -164.95; 32.59 -302.35 34.861]
*****
[390.12 53.034 296.01; 102.86 -15.358 87.843; -477.58 -46.895 -226.83]
*****
[163.72 117.25 -213.88; 154.44 133.69 -146.44; 259.4 320.89 -207.33]
*****
[45.593 21.837 -168.21; 166.08 137.2 -311.68; 77.686 141.76 -138.29]
*****
[-5.4906 -4.301 -34.579; 86.754 114.93 -469.75; 4.5217 59.763 -111.79]
*****
[5.054 -2.9375 -93.008; 132.55 108.53 -363.45; 20.453 91.849 -116.17]
*****
[74.973 65.682 -172.28; 157.06 152.09 -268.06; 119.14 209.27 -175.85]
*****


In [7]:
A = [-11.317 -10.12 2.8978; 45.466 86.037 -433.36; 0.028671 38.248 -62.274]

3×3 Array{Float64,2}:
 -11.317     -10.12      2.8978
  45.466      86.037  -433.36  
   0.028671   38.248   -62.274 

In [8]:
foo = eigen((A + A')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 
eigenvectors:
3×3 Array{Float64,2}:
 -0.0598078   0.996509   -0.0582456
  0.570036   -0.0138046  -0.821504 
  0.81944     0.0823344   0.56722  

In [9]:
foo.values

3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 

In [10]:
clamp.(foo.values,0, 10)

3-element Array{Float64,1}:
  0.0
  0.0
 10.0

In [93]:
#mutates out_Q and out_D_clamped
function get_tensor_warp!(T, e_min, e_max, out_Q, out_D_modified)
    S = (T + T')/2; #S is the symmetric part of T
    eigenDecomp = eigen(S);
    out_Q[:] = eigenDecomp.vectors
    #for lambda in eigenDecomp.values
        #TODO: reject values too close to zero?
     #   if lambda < 0 #reject negative
      #      return false;
      #  end
    #end
    #clamped_eigvals = clamp.(eigenDecomp.values,0,10)
    modified_eigvals = abs.(eigenDecomp.values) #no clamping
    out_D_modified[:] = Diagonal(modified_eigvals)
    return true;
end

get_tensor_warp! (generic function with 1 method)

In [94]:
Q = Array{Float64,2}(undef, 3,3)

3×3 Array{Float64,2}:
  0.802595  0.553009   0.223655
 -0.330697  0.724523  -0.604737
 -0.496468  0.411397   0.764377

In [95]:
D_clamped = Array{Float64,2}(undef, 3,3)

3×3 Array{Float64,2}:
  0.536719   0.605124   0.588012
 -0.588063  -0.231481   0.774983
 -0.605074   0.761736  -0.231611

In [96]:
fill!(Q,NaN)

3×3 Array{Float64,2}:
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

In [97]:
fill!(D_clamped,NaN)

3×3 Array{Float64,2}:
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

In [98]:
A

3×3 Array{Float64,2}:
 -11.317     -10.12      2.8978
  45.466      86.037  -433.36  
   0.028671   38.248   -62.274 

In [99]:
eigs_A_symm = eigen((A + A')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -199.80875227189026 
  -11.440926915527712
  223.69567918741873 
eigenvectors:
3×3 Array{Float64,2}:
 -0.0598078   0.996509   -0.0582456
  0.570036   -0.0138046  -0.821504 
  0.81944     0.0823344   0.56722  

In [100]:
get_tensor_warp!(A, -10, 10, Q, D_clamped)

true

In [101]:
B = [1 -1 0; -1 2 -1; 0 -1 1]

3×3 Array{Int64,2}:
  1  -1   0
 -1   2  -1
  0  -1   1

In [102]:
eigs_B_symm = eigen((B + B')/2)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 2.6645352591003757e-15
 1.0000000000000018    
 3.0                   
eigenvectors:
3×3 Array{Float64,2}:
 0.57735  -0.707107     -0.408248
 0.57735   9.42055e-16   0.816497
 0.57735   0.707107     -0.408248

In [103]:
get_tensor_warp!(B, -10, 10, Q, D_clamped)

true

In [104]:
Q

3×3 Array{Float64,2}:
 0.57735  -0.707107     -0.408248
 0.57735   9.42055e-16   0.816497
 0.57735   0.707107     -0.408248

In [105]:
D_clamped

3×3 Array{Float64,2}:
 2.66454e-15  0.0  0.0
 0.0          1.0  0.0
 0.0          0.0  3.0

In [106]:
[Q[i] for i in 1:9]

9-element Array{Float64,1}:
  0.5773502691896273   
  0.5773502691896258   
  0.5773502691896241   
 -0.7071067811865464   
  9.420554752102632e-16
  0.7071067811865488   
 -0.40824829046386313  
  0.8164965809277259   
 -0.40824829046386296  

In [107]:
# in_file: input CSV file
# out_file: CSV file with Q*D_clamped as the 3x3 matrix
# max_lambda: upper bound for eigenvalue after clamping
function process_tensors(in_file, out_file, max_lambda)
    tensor_df = readtable(in_file);
    out_handle = open(out_file, "w");
    
    for i in 1:size(tensor_df,1)
        curr_row = tensor_df[i,1:9];
        T_flat = [x for x in curr_row];
        T = transpose(reshape(T_flat, 3, 3))
        
        Q = Array{Float64,2}(undef, 3,3);
        fill!(Q,NaN);
        D_clamped = Array{Float64,2}(undef, 3,3);
        fill!(D_clamped,NaN);
        valid = get_tensor_warp!(T, 0, max_lambda, Q, D_clamped);
        
        if !valid
            continue;
        end
        
        #take a transpose here because Julia operates
        #in row major order
        A = transpose(Q*D_clamped); 
        #println(tensor_df[i,10]);
        out_s = @sprintf "%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f\n" A[1] A[2] A[3] A[4] A[5] A[6] A[7] A[8] A[9] tensor_df[i,10] tensor_df[i,11] tensor_df[i,12] ;       
        write(out_handle, out_s)
        #print(out_s);
    end
    
    close(out_handle);
end

process_tensors (generic function with 3 methods)

In [108]:
process_tensors("tensor_test.csv", "processed_tensor_test.csv", 10)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = process_tensors(::String, ::String, ::Int64) at In[107]:5
└ @ Main ./In[107]:5


In [109]:
process_tensors("tensor.csv", "processed_tensor.csv", 10)